In [ ]:
%load_ext autoreload

## Nearest Neighbor Hamiltonians

TEBD is an algorithm that works with nearest-neighbor interaction Hamiltonians. These are Hamiltonian that can be written as follows,
$$
H = \sum_{i=0}^{N-2} h_{i,i+1}. 
$$
with pairwise Hamiltonian terms $h_{i,i+1}$ between neighbors on a 1D system.

`NNHamiltonian` is an abstract class that provides the interface to gather these pairwise interactions $h_{i,i+1}.$ It does not assume anything about the Hamiltonian: it may change in time, it may be precomputed or it may be computed on the fly. Children classes take care of that.

In [ ]:
# file: mps/hamiltonians.py

import numpy as np
from numbers import Number
import scipy.sparse as sp

In [ ]:
# file: mps/hamiltonians.py

class NNHamiltonian(object):
    
    def __init__(self, size):
        #
        # Create a nearest-neighbor interaction Hamiltonian
        # of a given size, initially empty.
        #
        self.size = size
        self.constant = False

    def dimension(self, ndx):
        #
        # Return the dimension of the local Hilbert space
        #
        return 0
    
    def interaction_term(self, ndx, t=0.0):
        #
        # Return the interaction between sites (ndx,ndx+1)
        #
        return 0
    
    def tomatrix(self, t=0.0):
        """Return a sparse matrix representing the NNHamiltonian on the
        full Hilbert space."""
        
        # dleft is the dimension of the Hilbert space of sites 0 to (i-1)
        # both included
        dleft = 0
        # H is the Hamiltonian of sites 0 to i, this site included.
        H = 0 * sp.eye(self.dimension(0))
        for i in range(self.size-1):
            # We extend the existing Hamiltonian to cover site 'i+1'
            print(H.shape)
            H = sp.kron(H, sp.eye(self.dimension(i+1)))
            # We add now the interaction on the sites (i,i+1)
            print(self.interaction_term(i,t).shape)
            print(sp.kron(sp.eye(dleft if dleft else 1), self.interaction_term(i,t)).shape)
            H += sp.kron(sp.eye(dleft if dleft else 1), self.interaction_term(i,t))
            print(H.shape)
            # We extend the dimension covered
            dleft += self.dimension(i)

        return H

### Constant nearest-neighbor Hamiltonians

The first implementation is one that assumes (i) a constant Hamiltonian that (ii) can be decomposed into local terms and product between local operators
$$
H = \sum_i O_i + \sum_i \sum_n L^{(n)}_i \otimes R^{(n)}_{i+1}
$$

In order to construct the pairwise terms $h_{i,i+1}$, we will split the local terms equally among pairs. More precisely, the local term on the i-th site appears with equal weights on $h_{i-1,i}$ and $h_{i,i+1},$ as follows
$$
h_{i,i+1} = \sum_n L^{(n)}_i \otimes R^{(n)}_{i+1} +
\begin{cases}
O_i + \frac{1}{2} O_{i+1}, \text{ if  } i = 0 \\
\frac{1}{2} O_i + O_{i+1}, \text{ if  } i = N-2 \\
\frac{1}{2} O_i + \frac{1}{2} O_{i+1}, \text{ else  } 
\end{cases}.
$$

The function below computes the interaction terms $h_{i,i+1}$:

In [ ]:
# file: mps/hamiltonians.py


class ConstantNNHamiltonian(NNHamiltonian):

    def __init__(self, size, dimension):
        #
        # Create a nearest-neighbor interaction Hamiltonian with fixed
        # local terms and interactions.
        #
        #  - local_term: operators acting on each site (can be different for each site)
        #  - int_left, int_right: list of L and R operators (can be different for each site)
        #
        super(ConstantNNHamiltonian, self).__init__(size)
        self.constant = True
        self.int_left = [[] for i in range(size-1)]
        self.int_right = [[] for i in range(size-1)]
        self.interactions []*
        if isinstance(dimension, Number):
            dimension = [dimension] * size
        self.dimension_ = dimension

    def add_local_term(self, ndx, operator):
        #
        # Set the local term acting on the given site
        #
        if ndx == 0:
            self.add_interaction_term(ndx, operator, np.eye(self.dimension(1)))
        elif ndx == self.size-1:
            self.add_interaction_term(ndx-1, np.eye(self.dimension(ndx-1)), operator)
        else:
            self.add_interaction_term(ndx-1, np.eye(self.dimension(ndx-1)), 0.5*operator)
            self.add_interaction_term(ndx, 0.5*operator, np.eye(self.dimension(ndx+1)))

    def add_interaction_term(self, ndx, L, R):
        #
        # Add an interaction term $L \otimes R$ acting on sites 'ndx' and 'ndx+1'
        #
        # Add to int_left, int_right
        #
        # Update the self.interactions[ndx] term
        self.int_left[ndx].append(L)
        self.int_right[ndx].append(R)

    def dimension(self, ndx):
        return self.dimension_[ndx]

    def interaction_term(self, ndx, t=0.0):
        #for (L, R) in zip(self.int_left[ndx], self.int_right[ndx]):
            
        return sum([np.kron(L, R) for (L, R) in zip(self.int_left[ndx], self.int_right[ndx])])
    
    def constant(self):
        return True
            

A particular case would be a translationally invariant, constant Hamiltonian
$$H = \sum_i \left[O + \sum_n L^{(n)} \otimes R^{(n)}\right]_\text{site i}$$
which has the same local term $O$ on all sites, and the same interaction given by the product of $L^{(n)}$ left and $R^{(n)}$ right operators.

In [ ]:
# file: mps/hamiltonians.py

def make_ti_Hamiltonian(size, intL, intR, local_term=None):
    """Construct a translationally invariant, constant Hamiltonian with open
    boundaries and fixed interactions.
    
    Arguments:
    size        -- Number of sites in the model
    int_left    -- list of L (applied to site ndx) operators
    int_right   -- list of R (applied to site ndx + 1) operators
    local_term  -- operator acting on every site (optional)
    
    Returns:
    H           -- ConstantNNHamiltonian
    """
    if local_term is not None:
        dimension = len(local_term)
    else:
        dimension = len(intL[0])
    
    H = ConstantNNHamiltonian(size, dimension)
    H.local_term = local_term
    H.intL = intL
    H.intR = intR
    for ndx in range(size-1):
        for L,R in zip(H.intL, H.intR):
            H.add_interaction_term(ndx, L, R)
        if local_term is not None:
            H.add_local_term(ndx, local_term)
    return H

# Tests

In [ ]:
# file: mps/test/test_hamiltonians.py
from mps.hamiltonians import *


In [ ]:
# file: mps/test/test_TEBD.py
import unittest
import mps.state
import mps.tools
from mps.test.tools import *
from mps.tools import σx, σy, σz
import scipy.sparse as sp
import scipy.sparse.linalg
i2 = sp.eye(2)

class TestHamiltonians(unittest.TestCase):
    
    def test_nn_construct(self):
        H2 = ConstantNNHamiltonian(2, 2)
        H2.add_local_term(0, σx)
        M2 = H2.interaction_term(0)
        A2 = sp.kron(σx, i2)
        self.assertTrue(similar(M2, A2))
    
        H2 = ConstantNNHamiltonian(2, 2)
        H2.add_local_term(1, σy)
        M2 = H2.interaction_term(0)
        A2 = sp.kron(i2, σy)
        self.assertTrue(similar(M2, A2))

        H3 = ConstantNNHamiltonian(3, 2)
        H3.add_local_term(1, σy)
        M3 = H3.interaction_term(0)
        A3 = sp.kron(i2, 0.5*σy)
        self.assertTrue(similar(M3, A3))
        M3 = H3.interaction_term(1)
        A3 = sp.kron(0.5*σy, i2)
        self.assertTrue(similar(M3, A3))
    
    def test_sparse_matrix(self):
        H2 = ConstantNNHamiltonian(2, 2)
        H2.add_interaction_term(0, σz, σz)
        M2 = H2.tomatrix()
        A2 = sp.kron(σz,σz)
        self.assertTrue(similar(M2, A2))
        
        H2 = ConstantNNHamiltonian(2, 2)
        H2.add_local_term(0, 3.5*σx)
        M2 = H2.tomatrix()
        A2 = sp.kron(3.5*σx, i2)
        self.assertTrue(similar(M2, A2))
        
        H2 = ConstantNNHamiltonian(2, 2)
        H2.add_local_term(1, -2.5*σy)
        M2 = H2.tomatrix()
        A2 = sp.kron(i2, -2.5*σy)
        self.assertTrue(similar(M2, A2))
        
        H2 = ConstantNNHamiltonian(2, 2)
        H2.add_local_term(0, 3.5*σx)
        H2.add_local_term(1, -2.5*σy)
        H2.add_interaction_term(0, σz, σz)
        M2 = H2.tomatrix()
        A2 = sp.kron(i2, -2.5*σy) + sp.kron(σz,σz) + sp.kron(3.5*σx, i2)
        self.assertTrue(similar(M2, A2))

In [ ]:
%autoreload
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestHamiltonians'])
unittest.TextTestRunner(verbosity=2).run(suite1);